In [1]:
# Extensions
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload
%autoreload 2

# Imports
import torch
import pandas as pd
import numpy as np

# Configuration
ID = torch.Tensor([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
x = torch.randn(10, 4, 100)
x_rev_comp = torch.randn(10, 4, 100)
y = torch.randint(0, 2, (10, 1))
batch = (ID, x, x_rev_comp, y)

# Layers

In [324]:
from eugene.models.base import _layers as layers

## Activations

### Identity

In [325]:
layer = layers.Identity()

In [326]:
layer

Identity()

In [327]:
layer(x).shape

torch.Size([10, 4, 100])

### Exponential

In [328]:
layer = layers.Exponential()

In [329]:
layer

Exponential()

In [330]:
layer(x).shape

torch.Size([10, 4, 100])

## Convolutional

### Conv1D

In [331]:
layer = torch.nn.Conv1d(4, 8, 10)

In [332]:
layer

Conv1d(4, 8, kernel_size=(10,), stride=(1,))

In [333]:
layer.weight.shape

torch.Size([8, 4, 10])

In [334]:
layer(x).shape

torch.Size([10, 8, 91])

### BiConv1D

In [335]:
layer = layers.BiConv1D(4, 8, 10)

In [336]:
layer

BiConv1D(4, 8, kernel_size=10, stride=1, padding=same, dilation=1, groups=1, bias=True)

In [337]:
layer.weight.shape

torch.Size([8, 4, 10])

In [338]:
layer(x).shape

/Users/adamklie/Desktop/EUGENe/eugene/models/base/_layers.py:76: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at  /Users/distiller/project/pytorch/aten/src/ATen/native/Convolution.cpp:744.)
  x_fwd = F.conv1d(x, self.weight, stride=self.stride, padding=self.padding, dilation=self.dilation, groups=self.groups)


torch.Size([10, 8, 100])

## Pooling

## Recurrent

## Transformer

### MultiHeadAttention

In [345]:
layer = layers.MultiHeadAttention(
    input_dim=4,
    head_dim=10,
    num_heads=2,
)
layer

MultiHeadAttention(
  (qkv): Linear(in_features=4, out_features=60, bias=False)
  (softmax): Softmax(dim=-1)
  (dropout_layer): Dropout(p=0.0, inplace=False)
  (projection_layer): Sequential(
    (0): Linear(in_features=20, out_features=4, bias=True)
    (1): Dropout(p=0.0, inplace=False)
  )
)

In [346]:
layer(x.transpose(1,2), mask=None).shape

torch.Size([10, 100, 4])

## Normalization

## Wrappers

### Residual

In [347]:
conv_layer = torch.nn.Conv1d(4, 4, 10, padding="same")

In [348]:
layer = layers.Residual(conv_layer)

In [349]:
layer(x).shape

torch.Size([10, 4, 100])

## Gluers

### Flatten

In [52]:
layer = layers.Flatten()

In [55]:
layer(x).shape

torch.Size([10, 400])

### Unsqueeze

In [57]:
layer = layers.Unsqueeze(1)

In [59]:
layer(x).shape

torch.Size([10, 1, 4, 100])

### View

In [65]:
layer = layers.View((100, 4))


In [66]:
layer(x).shape

torch.Size([10, 100, 4])

## Miscallaneous

### Clip

In [68]:
layer = layers.Clip(0, 0.1)

In [70]:
layer(x).max()

tensor(0.1000)

# Blocks

In [252]:
from eugene.models.base import _blocks as blocks

## Conv1DBlock

In [387]:
conv1d_block = blocks.Conv1DBlock(
    input_len=100,
    input_channels=4,
    output_channels=4,
    conv_kernel=10,
    conv_type="biconv1d",
    conv_padding="same",
    pool_type="max",
    norm_type="layernorm",
    norm_dim=(4, 100),
    dropout_rate=0.5,
    order="conv-norm-act-dropout"
)
conv1d_block

Conv1DBlock(
  (layers): Sequential(
    (conv): BiConv1D(4, 4, kernel_size=10, stride=1, padding=same, dilation=1, groups=1, bias=True)
    (norm): LayerNorm((4, 100), eps=1e-05, elementwise_affine=True)
    (act): ReLU()
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [388]:
conv1d_block(x).shape

torch.Size([10, 4, 100])

In [389]:
conv1d_block.output_size

[4, 100]

In [390]:
layers.Residual(conv1d_block)(x).shape

torch.Size([10, 4, 100])

## DenseBlock

In [287]:
dense_block = blocks.DenseBlock(
    input_dim=400, 
    output_dim=1, 
    hidden_dims=[200, 100, 50, 10],
    activations="exponential",
    batchnorm=True,
    batchnorm_first=True, 
    dropout_rates=[0.1, None],
    biases=False
)
dense_block

DenseBlock(
  (layers): Sequential(
    (0): Linear(in_features=400, out_features=200, bias=False)
    (1): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Exponential()
    (3): Dropout(p=0.1, inplace=False)
    (4): Linear(in_features=200, out_features=100, bias=False)
    (5): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Exponential()
    (7): Linear(in_features=100, out_features=50, bias=False)
    (8): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): Exponential()
    (10): Linear(in_features=50, out_features=10, bias=False)
    (11): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): Exponential()
    (13): Linear(in_features=10, out_features=1, bias=False)
  )
)

In [289]:
dense_block(x.reshape(10, -1))

tensor([[-0.4820],
        [-1.0450],
        [-0.3707],
        [ 0.1420],
        [-1.7296],
        [-1.5558],
        [ 0.0736],
        [-0.3563],
        [-1.6115],
        [-0.5148]], grad_fn=<MmBackward0>)

## ReccurentBlock

In [290]:
recurrent_block = blocks.RecurrentBlock(
    input_dim=4,
    hidden_dim=10,
    num_layers=2,
    unit_type="lstm",
    bidirectional=True,
    dropout_rates=0.1,
    bias=False,
    batch_first=True
)
recurrent_block

RecurrentBlock(
  (layers): LSTM(4, 10, num_layers=2, bias=False, batch_first=True, dropout=0.1, bidirectional=True)
)

In [291]:
recurrent_block(x.transpose(1, 2))[0].shape

torch.Size([10, 100, 20])

# Towers

In [350]:
from eugene.models.base import _towers as towers

### Feed Forward Tower

In [354]:
tower = towers.Tower(
    input_size=400,
    block=torch.nn.Linear,
    repeats=3,
    dynamic_block_args={'in_features': [400, 200, 100], 'out_features': [200, 100, 10]},
)
tower

Tower(
  (blocks): Sequential(
    (linear_0): Linear(in_features=400, out_features=200, bias=True)
    (linear_1): Linear(in_features=200, out_features=100, bias=True)
    (linear_2): Linear(in_features=100, out_features=10, bias=True)
  )
)

In [355]:
tower.input_size, tower.output_size

(400, [10])

In [356]:
tower(x.reshape(10, -1)).shape

torch.Size([10, 10])

### Tower of Conv1D blocks

In [357]:
from eugene.models.base._blocks import Conv1DBlock

In [359]:
tower = towers.Tower(
    input_size=(4, 100),
    block=Conv1DBlock,
    repeats=3,
    static_block_args={'input_len': 100, 'conv_kernel': 3, 'conv_padding': 'same'},
    dynamic_block_args={'input_channels': [4, 10, 20], 'output_channels': [10, 20, 30]}
)
tower

Tower(
  (blocks): Sequential(
    (conv1dblock_0): Conv1DBlock(
      (layers): Sequential(
        (conv): Conv1d(4, 10, kernel_size=(3,), stride=(1,), padding=same)
        (norm): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): ReLU()
        (pool): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
      )
    )
    (conv1dblock_1): Conv1DBlock(
      (layers): Sequential(
        (conv): Conv1d(10, 20, kernel_size=(3,), stride=(1,), padding=same)
        (norm): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): ReLU()
        (pool): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
      )
    )
    (conv1dblock_2): Conv1DBlock(
      (layers): Sequential(
        (conv): Conv1d(20, 30, kernel_size=(3,), stride=(1,), padding=same)
        (norm): BatchNorm1d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act):

In [360]:
tower.input_size, tower.output_size

((4, 100), [30, 100])

In [361]:
tower(x).shape

torch.Size([10, 30, 100])

### Conv1DTower
Allows for more explicit definition of a tower of convolutions. Will be deprecated, but currently used to work with models.

In [365]:
conv1d_tower = towers.Conv1DTower(
    input_len=100,
    input_channels=4,
    conv_channels=[10, 10],
    conv_kernels=[3, 3],
    conv_strides=[1, 1],
    conv_dilations=[1, 1],
    conv_padding=["valid", "valid"],
    activations=[None, "relu"],
    pool_types="avg",
    pool_kernels=[1, 1],
    pool_strides=[1, 1],
    pool_padding=[0, 0],
    dropout_rates=[None, 0.5],
    batchnorm=True,
    batchnorm_first=True
)
conv1d_tower

Conv1DTower(
  (layers): Sequential(
    (0): Conv1d(4, 10, kernel_size=(3,), stride=(1,), padding=valid)
    (1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): AvgPool1d(kernel_size=1, stride=1, padding=(0,))
    (3): Conv1d(10, 10, kernel_size=(3,), stride=(1,), padding=valid)
    (4): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Dropout(p=0.5, inplace=False)
    (7): AvgPool1d(kernel_size=1, stride=1, padding=(0,))
  )
)

In [391]:
conv1d_tower(x).shape

torch.Size([10, 10, 96])

### BiConv1DTower

In [370]:
biconv1d_tower = towers.BiConv1DTower(
    filters = 10,
    kernel_size = 3,
    layers = 2
)
biconv1d_tower

BiConv1DTower(
  (kernels): ParameterList(
      (0): Parameter containing: [torch.FloatTensor of size 10x4x3]
      (1): Parameter containing: [torch.FloatTensor of size 10x10x3]
  )
  (biases): ParameterList(
      (0): Parameter containing: [torch.FloatTensor of size 10]
      (1): Parameter containing: [torch.FloatTensor of size 10]
  )
)

In [372]:
biconv1d_tower(x).shape

torch.Size([10, 10, 100])

# Base Model

## `SequenceModel`

In [3]:
from eugene.models.base._base_models import SequenceModel

# Models

## Sequence-to-function models 

### FCN

In [6]:
from eugene.models._sequence_to_function import FCN 

In [7]:
model = FCN(
    input_len=100,
    output_dim=10,
    task="regression",
    optimizer="adam",
    dense_kwargs={
        "hidden_dims": [50],
        "batchnorm": True,
    }
)
model

FCN(
  (train_metric): R2Score()
  (val_metric): R2Score()
  (test_metric): R2Score()
  (dense_block): DenseBlock(
    (layers): Sequential(
      (0): Linear(in_features=400, out_features=50, bias=True)
      (1): ReLU()
      (2): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): Linear(in_features=50, out_features=10, bias=True)
    )
  )
)

In [8]:
model(x).shape

torch.Size([10, 10])

### CNN

In [13]:
from eugene.models._sequence_to_function import CNN

In [22]:
model = CNN(
    input_len=100,
    output_dim=10,
    strand="ss",
    aggr="max",
    task="regression",
    optimizer="adam",
    conv_kwargs={
        "conv_channels": [10, 10],
        "conv_kernels": [5, 3],
        "activations": [],
        "pool_types": []
    }
)
model

CNN(
  (train_metric): R2Score()
  (val_metric): R2Score()
  (test_metric): R2Score()
  (conv1d_tower): Conv1DTower(
    (layers): Sequential(
      (0): Conv1d(4, 10, kernel_size=(5,), stride=(1,), padding=valid)
      (1): Conv1d(10, 10, kernel_size=(3,), stride=(1,), padding=valid)
    )
  )
  (dense_block): DenseBlock(
    (layers): Sequential(
      (0): Linear(in_features=940, out_features=10, bias=True)
    )
  )
)

In [24]:
model(x).shape

torch.Size([10, 10])

### RNN

In [26]:
from eugene.models._sequence_to_function import RNN

In [27]:
model = RNN(
    input_len=100,
    output_dim=10,
    strand="ds",
    aggr="max",
    task="regression",
    optimizer="adam",
    recurrent_kwargs={
        "hidden_dim": 10,
        "num_layers": 2,
        "bidirectional": True
    }
)
model

RNN(
  (train_metric): R2Score()
  (val_metric): R2Score()
  (test_metric): R2Score()
  (recurrent_block): RecurrentBlock(
    (layers): LSTM(4, 10, num_layers=2, batch_first=True, bidirectional=True)
  )
  (dense_block): DenseBlock(
    (layers): Sequential(
      (0): Linear(in_features=20, out_features=10, bias=True)
    )
  )
)

In [28]:
model(x.transpose(1, 2), x_rev_comp.transpose(1, 2)).shape

torch.Size([10, 10])

### Hybrid

In [29]:
from eugene.models._sequence_to_function import Hybrid

In [30]:
model = Hybrid(
    input_len=100,
    output_dim=10,
    strand="ss",
    aggr="max",
    task="regression",
    optimizer="adam",
    conv_kwargs={
        "conv_channels": [10, 10],
        "conv_kernels": [5, 3],
        "activations": "relu",
        "pool_types": "max"
    },
    recurrent_kwargs={
        "hidden_dim": 10,
        "num_layers": 10,
        "bidirectional": True
    }
)
model

Hybrid(
  (train_metric): R2Score()
  (val_metric): R2Score()
  (test_metric): R2Score()
  (conv1d_tower): Conv1DTower(
    (layers): Sequential(
      (0): Conv1d(4, 10, kernel_size=(5,), stride=(1,), padding=valid)
      (1): ReLU()
      (2): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
      (3): Conv1d(10, 10, kernel_size=(3,), stride=(1,), padding=valid)
      (4): ReLU()
      (5): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
    )
  )
  (recurrent_block): RecurrentBlock(
    (layers): LSTM(10, 10, num_layers=10, batch_first=True, bidirectional=True)
  )
  (dense_block): DenseBlock(
    (layers): Sequential(
      (0): Linear(in_features=20, out_features=10, bias=True)
    )
  )
)

In [31]:
model(x, x_rev_comp).shape

torch.Size([10, 10])

### TutorialCNN

In [33]:
from eugene.models._sequence_to_function import TutorialCNN

In [41]:
model = TutorialCNN(
    input_len=100,
    output_dim=10
)
model

TutorialCNN(
  (train_metric): R2Score()
  (val_metric): R2Score()
  (test_metric): R2Score()
  (conv1): Conv1d(4, 30, kernel_size=(21,), stride=(1,))
  (dense): Linear(in_features=30, out_features=10, bias=True)
)

In [35]:
model(x).shape

torch.Size([10, 10])

### Jores21CNN

In [36]:
from eugene.models._sequence_to_function import Jores21CNN

In [39]:
model = Jores21CNN(
    input_len=100,
    output_dim=10
)
model

Jores21CNN(
  (train_metric): R2Score()
  (val_metric): R2Score()
  (test_metric): R2Score()
  (biconv): BiConv1DTower(
    (kernels): ParameterList(
        (0): Parameter containing: [torch.FloatTensor of size 128x4x13]
        (1): Parameter containing: [torch.FloatTensor of size 128x128x13]
    )
    (biases): ParameterList(
        (0): Parameter containing: [torch.FloatTensor of size 128]
        (1): Parameter containing: [torch.FloatTensor of size 128]
    )
  )
  (conv): Conv1d(128, 128, kernel_size=(13,), stride=(1,), padding=same)
  (dropout): Dropout(p=0.15, inplace=False)
  (fc): Linear(in_features=12800, out_features=64, bias=True)
  (batchnorm): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=64, out_features=10, bias=True)
)

In [40]:
model(x).shape

torch.Size([10, 10])

### Kopp21CNN

In [42]:
from eugene.models._sequence_to_function import Kopp21CNN

In [44]:
model = Kopp21CNN(
    input_len=100,
    output_dim=10
)
model

/Users/adamklie/miniconda3/envs/eugene/lib/python3.7/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


Kopp21CNN(
  (train_metric): AUROC()
  (val_metric): AUROC()
  (test_metric): AUROC()
  (conv): Conv1d(4, 10, kernel_size=(11,), stride=(1,))
  (maxpool): MaxPool1d(kernel_size=30, stride=1, padding=0, dilation=1, ceil_mode=False)
  (batchnorm): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv1d(10, 8, kernel_size=(3,), stride=(1,))
  (batchnorm2): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear): Linear(in_features=8, out_features=10, bias=True)
)

In [45]:
model(x, x_rev_comp).shape

torch.Size([10, 10])

### FactorizedBasset

In [46]:
from eugene.models._sequence_to_function import FactorizedBasset

In [47]:
model = FactorizedBasset(input_len=100, output_dim=1)

/Users/adamklie/miniconda3/envs/eugene/lib/python3.7/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


In [48]:
model(x).shape

torch.Size([10, 1])

### ResidualBind 

In [49]:
from eugene.models._sequence_to_function import ResidualBind

In [51]:
model = ResidualBind(
    input_len=100,
    output_dim=1
)
model

ResidualBind(
  (train_metric): R2Score()
  (val_metric): R2Score()
  (test_metric): R2Score()
  (conv1d_tower): Conv1DTower(
    (layers): Sequential(
      (0): Conv1d(4, 96, kernel_size=(11,), stride=(1,), padding=valid)
      (1): BatchNorm1d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Dropout(p=0.1, inplace=False)
    )
  )
  (residual_block): Residual(
    (wrapped): Conv1DTower(
      (layers): Sequential(
        (0): Conv1d(96, 96, kernel_size=(3,), stride=(1,), padding=same)
        (1): BatchNorm1d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Dropout(p=0.1, inplace=False)
        (4): Conv1d(96, 96, kernel_size=(3,), stride=(1,), padding=same, dilation=(2,))
        (5): BatchNorm1d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (6): ReLU()
        (7): Dropout(p=0.1, inplace=False)
        (8): Conv1d(96, 96, kernel_size=(3,), stride=(1,)

In [52]:
model(x).shape

torch.Size([10, 1])

### DeepBind

In [53]:
from eugene.models._sequence_to_function import DeepBind

In [55]:
model = DeepBind(
    input_len=100,
    output_dim=1
)
model

DeepBind(
  (train_metric): R2Score()
  (val_metric): R2Score()
  (test_metric): R2Score()
  (conv1d_tower): Conv1DTower(
    (layers): Sequential(
      (0): Conv1d(4, 16, kernel_size=(16,), stride=(1,), padding=valid)
      (1): ReLU()
      (2): Dropout(p=0.25, inplace=False)
    )
  )
  (max_pool): MaxPool1d(kernel_size=85, stride=85, padding=0, dilation=1, ceil_mode=False)
  (avg_pool): AvgPool1d(kernel_size=(85,), stride=(85,), padding=(0,))
  (dense_block): DenseBlock(
    (layers): Sequential(
      (0): Linear(in_features=32, out_features=32, bias=True)
      (1): ReLU()
      (2): Dropout(p=0.25, inplace=False)
      (3): Linear(in_features=32, out_features=1, bias=True)
    )
  )
)

In [56]:
model(x).shape

torch.Size([10, 1])

### DeepSEA

In [57]:
from eugene.models._sequence_to_function import DeepSEA

In [58]:
model = DeepSEA(
    input_len=100,
    output_dim=1,
    conv_kwargs={
        "pool_types": ["avg", "max", None]
    }
)
model

DeepSEA(
  (train_metric): R2Score()
  (val_metric): R2Score()
  (test_metric): R2Score()
  (conv1d_tower): Conv1DTower(
    (layers): Sequential(
      (0): Conv1d(4, 320, kernel_size=(8,), stride=(1,), padding=valid)
      (1): ReLU()
      (2): Dropout(p=0.2, inplace=False)
      (3): AvgPool1d(kernel_size=4, stride=1, padding=(0,))
      (4): Conv1d(320, 480, kernel_size=(8,), stride=(1,), padding=valid)
      (5): ReLU()
      (6): Dropout(p=0.2, inplace=False)
      (7): MaxPool1d(kernel_size=4, stride=1, padding=0, dilation=1, ceil_mode=False)
      (8): Conv1d(480, 960, kernel_size=(8,), stride=(1,), padding=valid)
      (9): ReLU()
      (10): Dropout(p=0.5, inplace=False)
    )
  )
  (dense_block): DenseBlock(
    (layers): Sequential(
      (0): Linear(in_features=70080, out_features=925, bias=True)
      (1): ReLU()
      (2): Linear(in_features=925, out_features=1, bias=True)
    )
  )
)

In [59]:
model(x).shape

torch.Size([10, 1])

### Basset

In [60]:
from eugene.models._sequence_to_function import Basset

In [61]:
model = Basset(
    input_len=100,
    output_dim=1
)
model

/Users/adamklie/miniconda3/envs/eugene/lib/python3.7/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


Basset(
  (train_metric): AUROC()
  (val_metric): AUROC()
  (test_metric): AUROC()
  (conv1d_tower): Conv1DTower(
    (layers): Sequential(
      (0): Conv1d(4, 300, kernel_size=(19,), stride=(1,), padding=(9,))
      (1): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): MaxPool1d(kernel_size=3, stride=1, padding=0, dilation=1, ceil_mode=False)
      (4): Conv1d(300, 200, kernel_size=(11,), stride=(1,), padding=(5,))
      (5): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): ReLU()
      (7): MaxPool1d(kernel_size=4, stride=1, padding=0, dilation=1, ceil_mode=False)
      (8): Conv1d(200, 200, kernel_size=(7,), stride=(1,), padding=(3,))
      (9): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (10): ReLU()
      (11): MaxPool1d(kernel_size=4, stride=1, padding=0, dilation=1, ceil_mode=False)
    )
  )
  (dense_block): DenseBlock(
    (layers)

In [62]:
model(x).shape

torch.Size([10, 1])

### DanQ

In [63]:
from eugene.models._sequence_to_function import DanQ

In [64]:
model = DanQ(
    input_len=100,
    output_dim=1
)
model

DanQ(
  (train_metric): R2Score()
  (val_metric): R2Score()
  (test_metric): R2Score()
  (conv1d_tower): Conv1DTower(
    (layers): Sequential(
      (0): Conv1d(4, 320, kernel_size=(26,), stride=(1,), padding=same)
      (1): ReLU()
      (2): Dropout(p=0.2, inplace=False)
      (3): MaxPool1d(kernel_size=13, stride=1, padding=0, dilation=1, ceil_mode=False)
    )
  )
  (recurrent_block): RecurrentBlock(
    (layers): LSTM(320, 320, batch_first=True, bidirectional=True)
  )
  (dense_block): DenseBlock(
    (layers): Sequential(
      (0): Linear(in_features=640, out_features=925, bias=True)
      (1): ReLU()
      (2): Dropout(p=0.5, inplace=False)
      (3): Linear(in_features=925, out_features=1, bias=True)
    )
  )
)

In [65]:
model(x).shape

/Users/adamklie/miniconda3/envs/eugene/lib/python3.7/site-packages/torch/nn/modules/conv.py:299: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at  /Users/distiller/project/pytorch/aten/src/ATen/native/Convolution.cpp:744.)
  self.padding, self.dilation, self.groups)


torch.Size([10, 1])

### DeepSTARR

In [66]:
from eugene.models._sequence_to_function import DeepSTARR

In [67]:
model = DeepSTARR(
    input_len=100,
    output_dim=1
)
model

DeepSTARR(
  (train_metric): R2Score()
  (val_metric): R2Score()
  (test_metric): R2Score()
  (conv1d_tower): Conv1DTower(
    (layers): Sequential(
      (0): Conv1d(4, 246, kernel_size=(7,), stride=(1,), padding=same)
      (1): ReLU()
      (2): MaxPool1d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
      (3): BatchNorm1d(246, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (4): Conv1d(246, 60, kernel_size=(3,), stride=(1,), padding=same)
      (5): ReLU()
      (6): MaxPool1d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
      (7): BatchNorm1d(60, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (8): Conv1d(60, 60, kernel_size=(5,), stride=(1,), padding=same)
      (9): ReLU()
      (10): MaxPool1d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
      (11): BatchNorm1d(60, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (12): Conv1d(60, 120, kernel_size=(3,), str

In [68]:
model(x).shape

torch.Size([10, 1])

# Losses